In [ ]:
from numpy import genfromtxt
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import BatchNormalization
from matplotlib import pyplot
import numpy as np
from cnn_helper import cnn_classify

from google.colab import drive

In [ ]:
%matplotlib inline
!git clone -l -s https://github.com/Dhruv-Sabharwal/GAN_Face_Creator data
%cd data
!ls
drive.mount('/content/drive')

# Importing images and image preprocessing

In [ ]:
#Reading all the data from CelebA csv file into nparray
my_data = genfromtxt("celeba-dataset/list_attr_celeba.csv", delimiter=',',dtype='unicode')
list_delt=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41] # columns to be deleted

my_data=np.delete(my_data, list_delt, 1)  # deleting columns
my_data=np.delete(my_data, 0, 0)  # deleting first row (which contains the names of the columns)

# We are going to train the GAN separately for male and female images
sorted_array = my_data[np.argsort(my_data[:, 1])]   # sort by column [male = column 1]

i_names=[]
training_samples=35000

# Storing names of images we want in i_names
for i in range(training_samples):
  i_names.append(sorted_array[i][0])

# Storing images in images
import imageio
data_dir="celeba-dataset/img_align_celeba/img_align_celeba"
images=[]
for i in i_names:
  images.append(imageio.imread(data_dir + '/' + i))

# Converting to numpy array
images=np.asarray(images)

# scale images to preferred size
from skimage.transform import resize
images_list = list()
for image in images:
	# resize with nearest neighbor interpolation
	new_image = resize(image, (80,80), mode='constant')
	images_list.append(new_image)
 
# Converting to numpy arrays
images_list=np.asarray(images_list)
dataset=images_list  # dataset now contains all the images required to train our GAN model

# Defining GAN

In [ ]:
# Defining Discriminator
def define_discriminator(in_shape=(80,80,3)):
	model=Sequential()
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))

  # downsample to 40x40
	model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

  # downsample to 20x20
	model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

  # downsample to 10x10
	model.add(Conv2D(64, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

  # downsample to 5x5
	model.add(Conv2D(64, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier

	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))

	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model


# Defining Generator
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 5x5 feature maps
	n_nodes = 64 * 5 * 5
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((5, 5, 64)))

	# upsample to 10x10
	model.add(Conv2DTranspose(64, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	# upsample to 20x20
	model.add(Conv2DTranspose(64, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	# upsample to 40x40
	model.add(Conv2DTranspose(64, (7,7), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	# upsample to 80x80
	model.add(Conv2DTranspose(64, (9,9), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	# output layer 80x80x3
	model.add(Conv2D(3, (11,11), activation='tanh', padding='same'))
	return model


# Defining the GAN model
def define_gan(g_model, d_model):
	# making weights in the discriminator not trainable
	d_model.trainable = False
	model = Sequential()
	model.add(g_model)
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# select n real samples from the training dataset
def generate_real_samples(dataset, n):
  # generate n random numbers
  arr = randint(0, len(dataset)-1, n)
  # retrieve images from training dataset
  X =[]
  for i in range(len(arr)):
    X.append(dataset[arr[i]])
  X=np.asarray(X)
  y = ones((n, 1))  # The y label is always 1 for real samples 
  return X, y

# generate n points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
	# generate points in the latent space (gaussian distribution)
	x_input = randn(latent_dim * n)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n, latent_dim)
	return x_input

# use the generator to generate n synthetic images
def generate_fake_samples(g_model, latent_dim, n):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n)
	# predict outputs using generator i.e. generate synthetic images
	X = g_model.predict(x_input)
	y = zeros((n, 1))  # The y label is always 0 for fake samples 
	return X, y

# evaluate the discriminator and save periodically save the generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# get real samples from the dataset
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real samples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# use the generator to generate fake samples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save the generator model to Google-Drive
	filename = '/content/drive/My Drive/GAN-FEMALE3/generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=350, n_batch=128):
	bat_per_epo = len(dataset) // n_batch
	half_batch = n_batch // 2
	# iterate over n_epochs epochs
	for i in range(n_epochs):
	# iterate over bat_per_epo batches
		for j in range(bat_per_epo):
			# get real samples from the dataset
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
			# generate synthetic samples using the generator
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
			# generate latent points for input to the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# y labels as input to the GAN should be 1, so that the generator aims at getting better and fooling the discriminator
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss, g_acc = gan_model.train_on_batch(X_gan, y_gan)  # During GAN model training the discriminator weights are fixed
			# print loss on this batch
			print('>%d, %d/%d, d1=%.3f, a1=%.3f, d2=%.3f, a2=%.3f, g=%.3f, g_acc=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_acc1, d_loss2, d_acc2, g_loss, g_acc))
		# evaluate the model performance for every 10 epochs
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

# Training the GAN model

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

# Inference

In [ ]:
from keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randn

model_gen = load_model('generator_model_280.h5')

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space (Gaussian Distribution)
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the generator
	z_input = x_input.reshape(n_samples, latent_dim)
	return z_input

In [ ]:
latent_points = generate_latent_points(100, 100)
# generate images
X_gen  = model_gen.predict(latent_points)

In [ ]:
for i in range(100):
  plt.imshow(X_gen[i])
  plt.show()

In [ ]:
# generating faces with required facial features
gen_features = []
my_features = [1, 0 , 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]
while(True):
    latent_points = generate_latent_points(100, 1)
    # generate image
    X_gen  = model_gen.predict(latent_points)
    gen_features = cnn_classify(X_gen)
    if(gen_features==my_features):
        print("Got required features")
        break